# Phương pháp nghiên cứu khoa học

**Group 30:**

* Chung Kim Khánh - 23C11029
* Bùi Huỳnh Trung Nam - 23C11034

This repository (https://github.com/hukkelas/DSFD-Pytorch-Inference.git) includes:

* A High-Performance Pytorch Implementation of the paper **"DSFD: Dual Shot Face Detector" (CVPR 2019)**. Adapted from the original source code: https://github.com/Tencent/FaceDetection-DSFD.git
* Lightweight single-shot face detection from the paper **"RetinaFace: Single-stage Dense Face Localisation in the Wild"** adapted from https://github.com/biubug6/Pytorch_Retinaface.

In [2]:
import tensorflow as tf
print(tf.test.gpu_device_name())

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/device:GPU:0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9371548330285820940
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4956450807947361991
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
# Step 01. Check the colab enviroment

!cat /proc/meminfo

import torch

print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# additional info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

MemTotal:       13290480 kB
MemFree:         7779760 kB
MemAvailable:   11877140 kB
Buffers:          345188 kB
Cached:          3943484 kB
SwapCached:            0 kB
Active:           685824 kB
Inactive:        4395480 kB
Active(anon):       1112 kB
Inactive(anon):   803120 kB
Active(file):     684712 kB
Inactive(file):  3592360 kB
Unevictable:           4 kB
Mlocked:               4 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               480 kB
Writeback:             0 kB
AnonPages:        792552 kB
Mapped:           728424 kB
Shmem:             11548 kB
KReclaimable:     122496 kB
Slab:             169420 kB
SReclaimable:     122496 kB
SUnreclaim:        46924 kB
KernelStack:        4844 kB
PageTables:        13284 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6645240 kB
Committed_AS:    2534568 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       79172 kB
VmallocChunk:    

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:440: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Face Detection in Pytorch with DSFD and RetinaFace

## Improvements

### Difference from DSFD

For the original source code,


*   Removal of all unnecessary files for training / loading VGG models.
*   Improve the inference time by about 30x (from ~6s to 0.2) with rough estimates using time (Measured on a V100-32GB GPU).

The main improvements in inference time comes from:

*  Replacing non-maximum-suppression to a highly optimized torchvision version
* Refactoring 'init_priorsin' the SSD model to cache previous prior sizes (no need to generate this per forward pass).
* Refactoring the forward pass in Detect in 'utils.py' to perform confidence thresholding before non-maximum suppression.
* Minor changes in the forward pass to use pytorch 1.0 features.

### Difference from RetinaFace
For the original source code,
* Remove gradient computation for inference (torch.no_grad).
* Replacing non-maximum-suppression to a highly optimized torchvision version

In [5]:
# Commented out IPython magic to ensure Python compatibility.
%cd /content/gdrive/My\ Drive
%mkdir colab
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/
!rm -rf FaceDetection-DSFD
!git clone https://github.com/hukkelas/DSFD-Pytorch-Inference.git
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/DSFD-Pytorch-Inference

/content/gdrive/My Drive
mkdir: cannot create directory ‘colab’: File exists
/content/gdrive/My Drive/Colab Notebooks
fatal: destination path 'DSFD-Pytorch-Inference' already exists and is not an empty directory.
/content/gdrive/My Drive/Colab Notebooks/DSFD-Pytorch-Inference


In [6]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/DSFD-Pytorch-Inference
!python setup.py install

/content/gdrive/My Drive/Colab Notebooks/DSFD-Pytorch-Inference
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/buil

In [7]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/DSFD-Pytorch-Inference
!python test.py

/content/gdrive/My Drive/Colab Notebooks/DSFD-Pytorch-Inference
Downloading: "https://api.loke.aws.unit.no/dlr-gui-backend-resources-content/v2/contents/links/61be4ec7-8c11-4a4a-a9f4-827144e4ab4f0c2764c1-80a0-4083-bbfa-68419f889b80e4692358-979b-458e-97da-c1a1660b3314" to /root/.cache/torch/hub/checkpoints/61be4ec7-8c11-4a4a-a9f4-827144e4ab4f0c2764c1-80a0-4083-bbfa-68419f889b80e4692358-979b-458e-97da-c1a1660b3314
100% 459M/459M [00:23<00:00, 20.3MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Processing: images/0_Parade_Parade_0

In [12]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/DSFD-Pytorch-Inference
!python test_jpeg.py

/content/gdrive/My Drive/Colab Notebooks/DSFD-Pytorch-Inference
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Processing: images/istockphoto-861129134-612x612.jpeg
Detection time: 1.110
Processing: images/360_F_325765949_8p7suTucqzxv0omSiLRlhHaCbPRqcUxb.jpeg
Detection time: 0.150
Processing: images/joker-k6u5fleppdl2ajfb.jpeg
Detection time: 0.397
Processing: images/hoi-nghi-hoi-thao-1.jpeg
Detection time: 0.312
